<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/AgeGenderModelLoadingFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def load_age_gender_models():
    """Download and load the pre-trained age and gender models using direct URLs"""
    import os

    # Define more reliable direct URLs for the model files
    age_proto_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
    age_model_url = "https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/age_net.caffemodel"
    gender_proto_url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt"
    gender_model_url = "https://github.com/GilLevi/AgeGenderDeepLearning/raw/master/models/gender_net.caffemodel"

    # Define filenames
    age_proto_file = "age_deploy.prototxt"
    age_model_file = "age_net.caffemodel"
    gender_proto_file = "gender_deploy.prototxt"
    gender_model_file = "gender_net.caffemodel"

    # Use wget for more reliable downloads (with progress indicators)
    print("Downloading model files...")
    !wget -O {age_proto_file} {age_proto_url} --no-check-certificate
    !wget -O {age_model_file} {age_model_url} --no-check-certificate
    !wget -O {gender_proto_file} {gender_proto_url} --no-check-certificate
    !wget -O {gender_model_file} {gender_model_url} --no-check-certificate

    # Verify the files exist and have contents
    for file in [age_proto_file, age_model_file, gender_proto_file, gender_model_file]:
        if not os.path.exists(file) or os.path.getsize(file) == 0:
            raise FileNotFoundError(f"Failed to download or empty file: {file}")
        print(f"Successfully downloaded {file} ({os.path.getsize(file)} bytes)")

    # Load the models
    print("Loading models into OpenCV...")
    age_net = cv2.dnn.readNet(age_model_file, age_proto_file)
    gender_net = cv2.dnn.readNet(gender_model_file, gender_proto_file)

    print("Age and gender models loaded successfully")
    return age_net, gender_net

### Step 1: Import Libraries and Set Up Environment


In [2]:
!pip install ultralytics
!pip install roboflow
!pip install opencv-python

In [3]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from roboflow import Roboflow
import os

# Handle encoding
os.environ["PYTHONIOENCODING"] = "utf-8"

In [4]:
# Download your specific face detection dataset from Roboflow
rf = Roboflow(api_key="2IGtFaicFMGaMwb2mX8A")
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
version = project.version(25)
dataset = version.download("yolov11")

# Get the path to the downloaded dataset's YAML file
dataset_yaml = f"{dataset.location}/data.yaml"
print(f"Dataset YAML file located at: {dataset_yaml}")

# Load the YOLOv11 model
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # Build from YAML and transfer weights

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Face-Detection-25 in yolov11:: 100%|██████████| 8552/8552 [00:01<00:00, 6447.35it/s]


Dataset YAML file located at: /content/Face-Detection-25/data.yaml


100%|██████████| 5.35M/5.35M [00:00<00:00, 347MB/s]

Transferred 499/499 items from pretrained weights


In [5]:
def load_face_analytics_model():
    """Load a simplified face analytics model from OpenCV's DNN module"""

    # Define the path for downloading the models
    face_proto_path = "deploy.prototxt"
    face_model_path = "res10_300x300_ssd_iter_140000.caffemodel"

    # Download the model files if they don't exist
    if not os.path.exists(face_model_path):
        print("Downloading face detection model...")
        !wget -q -O {face_proto_path} https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
        !wget -q -O {face_model_path} https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/res10_300x300_ssd_iter_140000.caffemodel
        print("Download complete")

    # Load the face detection model
    face_net = cv2.dnn.readNet(face_model_path, face_proto_path)

    print("Face detection model loaded successfully")
    return face_net

In [6]:
def detect_faces(image_path, face_net=None, yolo_model=None):
    """Detect faces in an image using OpenCV DNN or YOLO model"""

    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image at {image_path}")
        return None

    # Convert to RGB for display with matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result_img = img_rgb.copy()
    h, w = img_rgb.shape[:2]

    faces = []

    # Method 1: Use OpenCV DNN if provided
    if face_net is not None:
        # Create a blob from the image
        blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], False, False)

        # Set the blob as input and perform inference
        face_net.setInput(blob)
        detections = face_net.forward()

        # Process detections
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filter based on confidence
            if confidence > 0.5:
                # Get the coordinates
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                x1, y1, x2, y2 = box.astype(int)

                # Ensure coordinates are valid
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(w, x2), min(h, y2)

                if x2 > x1 and y2 > y1:
                    faces.append((x1, y1, x2, y2))
                    cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Add a placeholder for age and gender (we'll skip actual prediction)
                    label = "Face Detected"
                    cv2.putText(result_img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Method 2: Use YOLO model if provided
    elif yolo_model is not None:
        # Perform face detection with YOLO
        results = yolo_model(img_rgb)

        # Process YOLO detections
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Get coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                # Filter based on confidence
                if conf > 0.5:
                    # Ensure coordinates are valid
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(w, x2), min(h, y2)

                    if x2 > x1 and y2 > y1:
                        faces.append((x1, y1, x2, y2))
                        cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

                        # Add confidence score as label
                        label = f"Face: {conf:.2f}"
                        cv2.putText(result_img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    print(f"Detected {len(faces)} faces")

    # Display result
    plt.figure(figsize=(12, 12))
    plt.imshow(result_img)
    plt.axis('off')
    plt.title(f"Detected {len(faces)} faces")
    plt.show()

    return result_img, faces